In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, History
from tensorflow.keras import regularizers
try:
    import shap
except ImportError:
    pass
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

# === Load dataset ===
df = pd.read_csv("Merged05.csv")
df.columns = df.columns.str.strip()

# Identify target column (assuming it contains 'label', 'attack', or 'class')
target_col = [col for col in df.columns if 'label' in col.lower() or 'attack' in col.lower() or 'class' in col.lower()][0]

# === Separate label and features ===
y = df[target_col]
X = df.drop(columns=[target_col]).select_dtypes(include=[np.number])

# Clean and align
df_clean = pd.concat([X, y], axis=1).replace([np.inf, -np.inf], np.nan).dropna()
X = df_clean.drop(columns=[target_col])
y = df_clean[target_col]

# Group rare classes (e.g., classes with < 100 samples)
class_counts = y.value_counts()
rare_classes = class_counts[class_counts < 100].index
y_grouped = y.copy()
for rare_class in rare_classes:
    y_grouped[y_grouped == rare_class] = "Rare"
print("Class distribution after grouping rare classes:\n", y_grouped.value_counts())

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_grouped)
num_classes = len(np.unique(y_encoded))

# Feature selection using correlation
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
X = X.drop(columns=to_drop)

# Normalize and clip
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = np.clip(X_scaled, -5, 5)

# Oversample rare classes with SMOTE
smote = SMOTE(random_state=42, sampling_strategy='not majority', k_neighbors=5)
X_scaled_resampled, y_encoded_resampled = smote.fit_resample(X_scaled, y_encoded)

# 5-fold cross-validation split
X_train_val, X_test, y_train_val, y_test = train_test_split(X_scaled_resampled, y_encoded_resampled, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, random_state=42)  # 10% of 80% for validation

# Compute class weights with adjustments
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_encoded_resampled), y=y_encoded_resampled)
class_weight_dict = {i: w for i, w in enumerate(class_weights)}  # Remove cap
poor_classes = ['DDOS-SYN_FLOOD', 'MIRAI-GREIP_FLOOD', 'RECON-OSSCAN', 'RECON-PORTSCAN']
for cls in poor_classes:
    idx = label_encoder.transform([cls])[0]
    if idx in class_weight_dict:
        class_weight_dict[idx] *= 1.5
print("Class weights:\n", class_weight_dict)

# Debug: Print shapes
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)

# === Focal Loss Implementation ===
def focal_loss(gamma=3.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        y_true = tf.one_hot(y_true, depth=num_classes)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * y_true * tf.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return focal_loss_fixed

# === Enhanced Feedforward Neural Network ===
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(num_classes, activation="softmax")
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=focal_loss(gamma=3.0, alpha=0.25),
              metrics=["accuracy"])
model.summary()

# Train with history callback to track accuracy
history = History()
early_stopping = EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=2, min_lr=1e-7)
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val),
          callbacks=[early_stopping, lr_scheduler, history],
          class_weight=class_weight_dict, verbose=1)

# === Random Forest Ensemble ===
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Combine predictions (simple voting)
y_pred_probs = model.predict(X_test)
y_pred_nn = np.argmax(y_pred_probs, axis=1)
ensemble_pred = np.round((y_pred_nn + rf_pred) / 2).astype(int)

# Evaluate
print("Neural Network Classification Report:\n", classification_report(y_test, y_pred_nn, target_names=label_encoder.classes_, zero_division=0))
print("Ensemble Classification Report:\n", classification_report(y_test, ensemble_pred, target_names=label_encoder.classes_, zero_division=0))

# === Generate SHAP Plot ===
if 'shap' in globals():
    shap_input = X_test[:100]
    print("SHAP input shape:", shap_input.shape)
    explainer = shap.KernelExplainer(lambda x: model.predict(x), shap_input)
    shap_values = explainer.shap_values(shap_input, nsamples=100)
    shap.summary_plot(shap_values, shap_input, feature_names=X.columns, plot_type="bar", show=False)
    plt.savefig("shap_summary.png")
    plt.close()
    print("SHAP summary plot saved as shap_summary.png")

# === Generate Training Accuracy Graph ===
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig("training_accuracy.png")
plt.close()
print("Training accuracy graph saved as training_accuracy.png")

# === Generate Confusion Matrix Plot ===
conf_matrix = confusion_matrix(y_test, ensemble_pred)
plt.figure(figsize=(12, 10))
plt.imshow(conf_matrix, interpolation="nearest", cmap=plt.cm.Blues)
plt.title("Ensemble Confusion Matrix")
plt.colorbar()
plt.ylabel("True label")
plt.xlabel("Predicted label")
tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, label_encoder.classes_, rotation=45)
plt.yticks(tick_marks, label_encoder.classes_)
plt.tight_layout()
plt.savefig("confusion_matrix_final.png")
plt.close()
print("Confusion matrix plot saved as confusion_matrix_final.png")

# === Reinforcement Learning for Mitigation ===
class IoTThreatMitigationEnv(gym.Env):
    def __init__(self, model, rf_model, X_data, y_true):
        super(IoTThreatMitigationEnv, self).__init__()
        self.model = model
        self.rf_model = rf_model
        self.X_data = X_data
        self.y_true = y_true
        self.current_step = 0
        self.action_space = gym.spaces.Discrete(3)  # 0: Block, 1: Throttle, 2: Allow
        self.observation_space = gym.spaces.Box(low=-5, high=5, shape=(self.X_data.shape[1] + num_classes,), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        X_sample = self.X_data[self.current_step]
        y_pred_nn = self.model.predict(X_sample.reshape(1, -1))
        y_pred_rf = self.rf_model.predict_proba(X_sample.reshape(1, -1))
        y_pred_onehot = np.zeros(num_classes)
        y_pred_onehot[np.argmax(y_pred_nn)] = 0.7  # Weight NN more
        y_pred_onehot[np.argmax(y_pred_rf)] += 0.3  # Weight RF less
        state = np.concatenate([X_sample, y_pred_onehot])
        return state

    def step(self, action):
        X_sample = self.X_data[self.current_step]
        y_pred_nn = self.model.predict(X_sample.reshape(1, -1))
        y_pred_rf = self.rf_model.predict_proba(X_sample.reshape(1, -1))
        y_pred_class = np.argmax(0.7 * y_pred_nn + 0.3 * y_pred_rf[0])
        y_true_class = self.y_true[self.current_step]

        # Reward logic
        reward = 0
        if y_true_class != 0:  # Not BENIGN
            if action in [0, 1]:  # Block or Throttle
                reward = 1  # Successful mitigation
            else:
                reward = -2  # Strong penalty for missed attack
        else:  # BENIGN
            if action == 2:  # Allow
                reward = 1  # Correctly allowed benign traffic
            else:
                reward = -1  # Penalty for blocking benign

        self.current_step += 1
        done = self.current_step >= len(self.X_data)
        next_X = self.X_data[self.current_step] if not done else X_sample
        next_y_pred_nn = self.model.predict(next_X.reshape(1, -1))
        next_y_pred_rf = self.rf_model.predict_proba(next_X.reshape(1, -1))
        next_y_pred_onehot = np.zeros(num_classes)
        next_y_pred_onehot[np.argmax(next_y_pred_nn)] = 0.7
        next_y_pred_onehot[np.argmax(next_y_pred_rf)] += 0.3
        next_state = np.concatenate([next_X, next_y_pred_onehot])
        return next_state, reward, done, {}

    def render(self):
        pass

# Initialize and train RL model
env = IoTThreatMitigationEnv(model, rf_model, X_test, y_test)
check_env(env)
rl_model = PPO("MlpPolicy", env, verbose=1)
rl_model.learn(total_timesteps=10000)
rl_model.save("iot_mitigation_ppo")
print("RL model trained and saved as iot_mitigation_ppo")

ModuleNotFoundError: No module named 'stable_baselines3'